In [189]:
import pandas as pd 
import ast
import numpy as np 
import string
import time
from geopy.geocoders import Nominatim

In [190]:
da = pd.read_csv("driver_activity.csv") # Import Driver Activity
dq = pd.read_csv("driver_quality.csv") # Import drivers_quality 
dp = pd.read_csv("payments_order.csv") # Import payment order
tl = pd.read_csv("trip_location.csv") # Import trip Location 

In [ ]:
# function for extracting coordinates of pick up and drop-off location , delay the funtion by one second to avoid trace error.. 

def get_lan_lng_geopy(adress):
    geolocator = Nominatim(user_agent= "Germany")
    time.sleep(1)
    location = geolocator.geocode(adress)
    if location: 
        return location.latitude, location.longitude
    else: 
        return None

In [ ]:
# applying the function for extracting geo location. 
abhol_geo_location = []
ziel_geo_location = []
for i ,j in zip(tl["Abholadresse"], tl["Zieladresse"]):
     abhol_geo_location.append(get_lan_lng_geopy(i))
     ziel_geo_location.append(get_lan_lng_geopy(j))

In [ ]:
# appending the geo location stored in list into the dataframe 
tl["Abhol_geo"] = abhol_geo_location 
tl["Ziel_geo"] = ziel_geo_location

In [191]:
# funtion to transfrom the online time from DD:HH:MM to total hours online
def in_to_hours(x):
    a = (x.split(":"))
    return round(int(a[0])*24 + int(a[1]) + (int(a[2])/60))



In [192]:
# apply funtion 
da["Online"] = da["Zeit online (Tage: Stunden: Minuten)"].apply(in_to_hours)

In [193]:
#defina a function to reverse the type of an entry into its original format from a String, althouh need to repeat again once saved as csv
def redo_type(x):
    #check if x is NAN using pandas 
    if pd.notna(x):
        try:
            return ast.literal_eval(x)
        except(ValueError, SyntaxError):
            return x
    else:
        return np.NAN

In [194]:
#Implementing into the geo data, which is essentially a Tuple datatype but recorded as string
tl['Abhol_geo'] = tl['Abhol_geo'].apply(redo_type)
tl["Ziel_geo"] = tl["Ziel_geo"].apply(redo_type)
tl["Date"]= pd.to_datetime(tl["Zeitpunkt der Fahrtbestellung"]).dt.strftime('%d.%b')

In [195]:
#renaming the column 
dq.columns.values[8] = "Fahrer_Bewertung"

In [196]:
# renaming the column
dp.columns.values[10] = "Fahrt_Umsatz"
dp.columns.values[26] = "Aktion"

In [198]:
# joining the tables based on entity relation.
df = pd.merge(tl,dq, on="Fahrer-UUID", how="left")
df = pd.merge(df,da, on="Fahrer-UUID", how="left")
df = pd.merge(df,da, on="Fahrer-UUID", how="left")
df = pd.merge(df,dp[["Fahrt_Umsatz","Aktion","Fahrt-UUID"]], on="Fahrt-UUID", how="outer")

In [201]:
# selecting only relevant columns..
data = df[["Vorname des Fahrers_x","Nachname des Fahrers_x","Abholadresse", "Zieladresse", 
    "Fahrtdistanz", "Abhol_geo", "Ziel_geo", "Abgeschlossene Fahrten_x", "Annahmerate",
    "Fahrer_Bewertung",'Online', 'Fahrt_Umsatz', 'Aktion', "Date"]]

In [204]:
# renaming columns 
data.columns.values[0] = "First_Name"
data.columns.values[1] = "last_Name"
data.columns.values[7] = "Abgeschlossene_Fahrten"

In [ ]:
# fullname, to avoid duplicates 
data["Drivers_Name"] = data["First_Name"] + " " + data["last_Name"]

In [206]:
# introducing pseudo Names for each Driver.. 
pair = {}
for i,j in zip(data["Drivers_Name"].unique(), [i.upper() for i in string.ascii_letters]):
    pair[i] = j + "xter" + " " + j +"ustermann"

In [207]:
# dictionary to dataframe 
paired = pd.DataFrame(list(pair.items()), columns=['Drivers_Name', 'Pseudo_Drivers_Name']).reset_index(drop=True)

In [208]:
# merging into the data, something like vlook in excel 
data = pd.merge(data, paired, on = "Drivers_Name", how = "left"  )

In [209]:
# removing first name and last name of Drivers 
data = data.iloc[:,2:]

In [210]:
# removing the full name of Drivers 
data.drop("Drivers_Name", axis = 1, inplace = True)

In [211]:
# Save for futher use 
# data.to_csv("data.csv", index=False)